In [2]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data


data104924  data104925


In [ ]:

%cd ~/work/
!git clone https://hub.fastgit.org/PaddlePaddle/PaddleVideo.git
!ls /home/aistudio/work

In [1]:

!python3.7 -m pip install --upgrade pip
!python3.7 -m pip install --upgrade -r /home/aistudio/work/PaddleVideo/requirements.txt

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 1.7MB 14.4MB/s eta 0:00:01
  Found existing installation: pip 19.2.3
    Uninstalling pip-19.2.3:
      Successfully uninstalled pip-19.2.3
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 15.7 MB 55.5 MB/s            
     |████████████████████████████████| 11.3 MB 47.6 MB/s            
     |████████████████████████████████| 76 kB 13.5 MB/s            
     |████████████████████████████████| 262 kB 71.4 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 28.2 MB 66.0 MB/s            
     |████████████████████████████████| 11.8 MB 48.7 MB/s            
     |████████████████████████████████| 37.2 MB 55.8 MB/s            
  Created wheel for pyyaml: filename=PyYAML-4.2b2-cp37-cp37m-linux_x86_64.whl size=43031 sha256=bdbaee5751c7a17fff991fed6c60682c2f661ac94bb4481f955683e661cd91bc
  Stored 

In [2]:
%cd ~/work/PaddleVideo/

/home/aistudio/work/PaddleVideo


In [4]:

##############################split train set and validation set
import numpy as np
original_data = np.load('/home/aistudio/data/data104925/train_data.npy')
original_label = np.load('/home/aistudio/data/data104925/train_label.npy')
permuted_idx = np.random.permutation(np.arange(len(original_label)))
train_idx = permuted_idx[150:]
train_data = original_data[train_idx]
train_label = original_label[train_idx]
valid_idx = permuted_idx[:150]
valid_data = original_data[valid_idx]
valid_label = original_label[valid_idx]
data_path = '/home/aistudio/split_dataset'
import os
if not os.path.isdir(data_path):
        os.makedirs(data_path)
np.save('/home/aistudio/split_dataset/train_data',train_data)
np.save('/home/aistudio/split_dataset/train_label',train_label)
np.save('/home/aistudio/split_dataset/valid_data',valid_data)
np.save('/home/aistudio/split_dataset/valid_label',valid_label)
print(train_data.shape,valid_data.shape)

(2772, 3, 2500, 25, 1) (150, 3, 2500, 25, 1)


In [1]:
################generate bone data
# reference:
# @inproceedings{2sagcn2019cvpr,  
#       title     = {Two-Stream Adaptive Graph Convolutional Networks for Skeleton-Based Action Recognition},  
#       author    = {Lei Shi and Yifan Zhang and Jian Cheng and Hanqing Lu},  
#       booktitle = {CVPR},  
#       year      = {2019},  
# }

# @article{shi_skeleton-based_2019,
#     title = {Skeleton-{Based} {Action} {Recognition} with {Multi}-{Stream} {Adaptive} {Graph} {Convolutional} {Networks}},
#     journal = {arXiv:1912.06971 [cs]},
#     author = {Shi, Lei and Zhang, Yifan and Cheng, Jian and LU, Hanqing},
#     month = dec,
#     year = {2019},
# }

import numpy as np
from numpy.lib.format import open_memmap
from tqdm import tqdm

paris= [(1, 8), (0, 1), (15, 0), (17, 15), (16, 0),
                             (18, 16), (5, 1), (6, 5), (7, 6), (2, 1), (3, 2),
                             (4, 3), (9, 8), (10, 9), (11, 10), (24, 11),
                             (22, 11), (23, 22), (12, 8), (13, 12), (14, 13),
                             (21, 14), (19, 14), (20, 19)]
################train and valid############
datasets=('train','valid')
for dataset in datasets:

    data = np.load('split_dataset/{}_data.npy'.format(dataset))
    N, C, T, V, M = data.shape
    fp_sp = open_memmap(
        'split_dataset/{}_data_bone.npy'.format(dataset),
        dtype='float32',
        mode='w+',
        shape=(N, 3, T, V, M))

    fp_sp[:, :C, :, :, :] = data
    for v1, v2 in tqdm(paris):
        fp_sp[:, :, :, v1, :] = data[:, :, :, v1, :] - data[:, :, :, v2, :]

###############test##################
# data = np.load('data/data117808/test_B_data.npy')
# N, C, T, V, M = data.shape
# fp_sp = open_memmap(
#     'data/data117808/test_B_data_bone.npy',
#     dtype='float32',
#     mode='w+',
#     shape=(N, 3, T, V, M))

# fp_sp[:, :C, :, :, :] = data
# for v1, v2 in tqdm(paris):
#     fp_sp[:, :, :, v1, :] = data[:, :, :, v1, :] - data[:, :, :, v2, :]

100%|██████████| 24/24 [00:00<00:00, 27.83it/s]


In [6]:
#训练模型
!python3.7 main.py -c configs/recognition/aagcn/aagcn_fsd.yaml --validate

[11/18 12:59:56] DALI is not installed, you can improve performance if use DALI
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/yaml/constructor.py:126: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  if not isinstance(key, collections.Hashable):
[11/18 12:59:56] DATASET : 
[11/18 12:59:56]     batch_size : 32
[11/18 12:59:56]     num_workers : 4
[11/18 12:59:56]     test : 
[11/18 12:59:56]         file_path : /home/aistudio/data/data104924/test_A_data.npy
[11/18 12:59:56]         format : SkeletonDataset
[11/18 12:59:56]         test_mode : True
[11/18 12:59:56]     test_batch_size : 1
[11/18 12:59:56]     test_num_workers : 0
[11/18 12:59:56]     train : 
[11/18 12:59:56]         file_path : /home/aistudio/split_dataset/train_data.npy
[11/18 12:59:56]         format : SkeletonDataset
[11/18 12:59:56]         label_path : /home/aistudio/split_dataset/train_label.np

In [3]:
#测试模型
!python3.7 main.py --test -c configs/recognition/aagcn/aagcn_fsd.yaml -w output/AAGCN-Joint/AAGCN-Joint_best.pdparams

[11/23 21:02:24] DALI is not installed, you can improve performance if use DALI
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/yaml/constructor.py:126: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  if not isinstance(key, collections.Hashable):
[11/23 21:02:24] DATASET : 
[11/23 21:02:24]     batch_size : 32
[11/23 21:02:24]     num_workers : 4
[11/23 21:02:24]     test : 
[11/23 21:02:24]         file_path : /home/aistudio/data/data117808/test_B_data.npy
[11/23 21:02:24]         format : SkeletonDataset
[11/23 21:02:24]         test_mode : True
[11/23 21:02:24]     test_batch_size : 1
[11/23 21:02:24]     test_num_workers : 0
[11/23 21:02:24]     train : 
[11/23 21:02:24]         file_path : /home/aistudio/split_dataset/train_data.npy
[11/23 21:02:24]         format : SkeletonDataset
[11/23 21:02:24]         label_path : /home/aistudio/split_dataset/train_label.np

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 